In [1]:
import pandas as pd

In [2]:
ah_k = pd.read_csv(r"data\AH_premium_analysis.csv", encoding="utf-8-sig")
ah_k.columns

Index(['ts_code_A', 'ts_code_HK', 'trade_date', 'name', 'p_a_adj', 'p_hk_adj',
       'p_hk', 'p_a', 'adj_factor_HK', 'adj_factor_A', 'adj_base_HK',
       'adj_base_A', 'hk2cn', 'AH_premium_adj', 'AH_premium'],
      dtype='object')

In [3]:
# -------- 0) 前置：确保日期列已是 datetime ----------
ah_k["trade_date"] = pd.to_datetime(ah_k["trade_date"].astype(str))

# -------- 1) 生成按月快照（取每月最后一个交易日的溢价率） ----------
ah_k_sorted = ah_k.sort_values("trade_date")  # 先按时间升序
prem_col = "AH_premium_adj"  # 如果换列名记得同步

# level 0: ts_code_A, level 1: month
month_series = (
    ah_k_sorted.assign(month=lambda df: df["trade_date"].dt.to_period("M"))
    .groupby(["ts_code_A", "month"])[prem_col]
    .last()  # 当月最后一个交易日
)

# -------- 2) 计算溢价率变化倍数：本月 / 上月 ----------
month_ratio = month_series.groupby(level=0).apply(lambda s: s / s.shift(1))

# -------- 3) 统计每只股票的方差（skipna=True 默认忽略首月 NaN） ----------
var_df = month_ratio.groupby(level=0).var().reset_index()
var_df.columns = ["ts_code_A", "var_monthly_premium_change"]  # ← 直接覆写列名

# -------- 4) 辅助列：加入中文股票简称，方便读表 ----------
name_map = ah_k_sorted[["ts_code_A", "name"]].drop_duplicates()
var_df = var_df.merge(name_map, on="ts_code_A", how="left")

# -------- 5) 按方差从高到低排序，看看哪些 AH 价差最“躁动” ----------
var_df = var_df.sort_values("var_monthly_premium_change", ascending=False)

var_df.sort_values(by="var_monthly_premium_change")
var_df.to_csv(
    "data/ah_monthly_premium_change_var.csv", index=False, encoding="utf-8-sig"
)

var_df.head(10)  # 想看前 10 条就 print / display 一下

,ts_code_A,var_monthly_premium_change,name
62,601328.SH,1617.804180,交通银行
50,600988.SH,51.186062,赤峰黄金
26,002948.SZ,36.024135,青岛银行
1,000063.SZ,27.066210,中兴通讯
65,601398.SH,24.294193,工商银行
70,601607.SH,21.991603,上海医药
10,000951.SZ,21.746591,中国重汽
58,601288.SH,19.629926,农业银行
34,600036.SH,18.874177,招商银行
33,600030.SH,14.021963,中信证券
